Import packages

In [1]:
import pyfits
import numpy as np

shell script to download data - do this through python instead

#!/bin/bash

targdir=a1835_xmm
com="wget -nd -P $targdir"
dir=http://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0098010101/PPS/
files=( P0098010101M2U009IMAGE_3000.FTZ P0098010101M2U009EXPMAP3000.FTZ  P0098010101M2X000BKGMAP3000.FTZ )

for file in ${files[@]}; do
    [ -e $targdir/$file ] || $com $dir$file
done


Read in the XMM MOS2 image

In [3]:
imfits = pyfits.open('a1835_xmm/P0098010101M2U009IMAGE_3000.FTZ')
imfits.info()

Filename: a1835_xmm/P0098010101M2U009IMAGE_3000.FTZ
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU     262   (648, 648)   int32
1    GTI00006    BinTableHDU     29   15R x 2C     [D, D]
2    GTI00106    BinTableHDU     29   15R x 2C     [D, D]
3    GTI00206    BinTableHDU     29   16R x 2C     [D, D]
4    GTI00306    BinTableHDU     29   15R x 2C     [D, D]
5    GTI00406    BinTableHDU     29   15R x 2C     [D, D]
6    GTI00506    BinTableHDU     29   15R x 2C     [D, D]
7    GTI00606    BinTableHDU     29   15R x 2C     [D, D]


The image is saved as integer type. If we need it to be floating point for some reason, we need to cast it:
im = imfits[0].data.astype('np.float32')
Note that this (probably?) prevents us from using the pyfits "writeto" method to save any changes.
Assuming the integer type is ok, just get a pointer to the image data.

In [4]:
im = imfits[0].data

Also read in the model background and exposure maps. (We may not need these immediately.) They are both float32 type.

In [10]:
bkfits = pyfits.open('a1835_xmm/P0098010101M2X000BKGMAP3000.FTZ')
bkfits.info()

Filename: a1835_xmm/P0098010101M2X000BKGMAP3000.FTZ
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      77   (648, 648)   float32


In [11]:
exfits = pyfits.open('a1835_xmm/P0098010101M2U009EXPMAP3000.FTZ')
exfits.info()

Filename: a1835_xmm/P0098010101M2U009EXPMAP3000.FTZ
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      63   (648, 648)   float32


## Exercise

Let's look at the data image, with `ds9`. What do you think is going on? Make a list with your neighbor, and report back in 5 minutes.

In [9]:
datafile = 'a1835_xmm/P0098010101M2U009IMAGE_3000.FTZ'
!ds9 -log "$datafile"

In [27]:
print im, len(im),im.shape,im[324,324],np.max(im),im.argmax(),type(im),im[348,328], im.T

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] 648 (648, 648) 0 223 225832 <type 'numpy.ndarray'> 223 [[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


todo: figure out whether the index ordering of the ndarray matches the usual image coordinates. nope, they're reversed (and also indexed from zero)